## Import modules

In [64]:
import pytesseract as pst
import PIL.Image as pi
import cv2
from pytesseract import Output
from wand.image import Image as wii
from pdf2image import convert_from_path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import string
import IPython.display as Ipd
from glob import glob as gg
from tqdm import tqdm

# Data preporition

### Returning text from image

In [65]:
# Text recognition configuration
MY_CONFIG = r"--psm 3 --oem 3 -l eng" # --psm 3, 6

# Images link
PNG_FILES = gg('./png/*.png')

# Files that we use for
test_files_ix = [2, 7, 9]

In [66]:
def doc_ix(filename):
    filename = filename[filename.index('\\'):]
    
    for char in filename:
        if char.isnumeric():
            first_digit_ix = filename.index(char)
            filename = filename[first_digit_ix:]

            for char in filename:
                if not char.isnumeric():
                    last_digit_ix = filename.index(char)
                    filename = filename[:last_digit_ix]
                    
                    return int(filename)

In [67]:
def img_ix(filename):
    filename = filename[::-1]

    for char in filename:
        if char.isnumeric():
            first_digit_ix = filename.index(char)
            filename = filename[first_digit_ix:]

            for char in filename:
                if not char.isnumeric():
                    last_digit_ix = filename.index(char)
                    filename = filename[:last_digit_ix]
                    filename = filename[::-1]
                    
                    return int(filename)

In [68]:
text_list_doc1 = []
text_list_doc2 = []
text_list_doc3 = []

header_list = []
footer_list = []

In [69]:
def get_header(list1, list2, have_a_header):
    new_list = []
    
    for word in list1:
        if word in list2:
            new_list.append(word)
            have_a_header = True
        else:
            break

    return new_list

In [70]:
def get_footer(list1, list2, have_a_footer):
    new_list = []

    # for word in list1:
    #     if not word.isalpha():
    #         list1.remove(word)
    
    # for word in list2:
    #     if not word.isalpha():
    #         list1.remove(word)

    list1.reverse()
    list2.reverse()

    for word in list1:
        if word in list2:
            new_list.append(word)
            have_a_footer = True
        else:
            break

    return new_list

In [71]:
train_text1 = []
train_text2 = []
train_text3 = []
train_text4 = []
train_text5 = []
train_text6 = []

In [72]:
prev_page = []
have_a_header = False
have_a_footer = False

for file in PNG_FILES:
    test_text = pst.image_to_string(pi.open(file), config = MY_CONFIG)
    page = test_text.split()
    
    if (doc_ix(file) in test_files_ix):
        if doc_ix(file) == test_files_ix[0] and img_ix(file) >= 4:
            text_list_doc1 += page
        elif doc_ix(file) == test_files_ix[1] and img_ix(file) >= 20:
            text_list_doc2 += page
        elif doc_ix(file) == test_files_ix[2] and (img_ix(file) >= 6 and img_ix(file) <= 14):
            text_list_doc3 += page
        
        if prev_page and not have_a_header:
            header_list += get_header(prev_page, page, have_a_header)
        
        if prev_page and not have_a_footer:
            footer_list += get_footer(prev_page, page, have_a_footer)
        
        prev_page = page
        
    else:
        if doc_ix(file) == 1 and img_ix(file) <= 18:
            train_text1 += page
        elif doc_ix(file) == 3:
            train_text2 += page
        elif doc_ix(file) == 4 and (img_ix(file) == 6 or img_ix(file) == 7 or (img_ix(file) >= 12 and img_ix(file) <= 14) or img_ix(file) >= 23):
            train_text3 += page
        elif doc_ix(file) == 5 and img_ix(file) >= 6:
            train_text4 += page
        elif doc_ix(file) == 6:
            train_text5 += page
        elif doc_ix(file) == 8 and img_ix(file) >= 9:
            train_text6 += page

In [73]:
text_list_doc = text_list_doc1 + text_list_doc2 + text_list_doc3

text_list_doc = [item for item in text_list_doc if item not in header_list]
text_list_doc = [item for item in text_list_doc if item not in footer_list]

In [74]:
origin_text_elements = len(text_list_doc)
origin_text_elements

7559

## Fuctions

### Drop Punctuation and Save words in Uppercase

In [75]:
def add_id(val, idx):
    return (idx, val)

In [76]:
stop_words = ['the', 'of', 'and', 'in', 'to', 'for', 'is',
              'as', 'with', 'on', 'are', 'this', 'that', 'it', 
              'by', 'an', 'be', 'was', 'has', 'such', 'from',
              'he', 'all', 'use', 'new', 'its', 'which', 'their',
              'can', 'at', 'where', 'when', 'not', 'if', 'there',
              'or', 'you', 'they', 'each', 'have', 'them', 'these',
              'out', 'then', 'etc', 'time', 'also', 'we', 'thus',
              'about', 'should', 'lets', 'will']

In [77]:
def func(any_list):
    lower_list = []
    lower_id = []
    
    upper_list = []
    upper_id = []

    name_list = []
    name_id = []

    id = 0
    
    for word in any_list:
        word = word.translate(str.maketrans('', '', string.punctuation))

        if len(word) > 1 and word.lower() not in stop_words:
            if word.isupper():
                upper_list.append(word)
                upper_id.append(id)

            elif word.islower():
                lower_list.append(word)
                lower_id.append(id)
            
            elif word.isalpha():
                name_list.append(word)
                name_id.append(id)
            
            id += 1

    for word in name_list:
        if word.lower() in name_list:
            name_list.remove(word)
    
    for word in upper_list:
        if word.lower() in upper_list:
            upper_list.remove(word)

    upper_map = list(map(add_id, upper_list, upper_id))
    lower_map = list(map(add_id, lower_list, lower_id))
    name_map = list(map(add_id, name_list, name_id))

    return upper_map, name_map, lower_map

In [78]:
uppers, names, lowers = func(text_list_doc)
all_words = uppers + names + lowers

In [79]:
uppers1, names1, lowers1 = func(train_text1)
uppers2, names2, lowers2 = func(train_text2)
uppers3, names3, lowers3 = func(train_text3)
uppers4, names4, lowers4 = func(train_text4)
uppers5, names5, lowers5 = func(train_text5)
uppers6, names6, lowers6 = func(train_text6)

In [80]:
train_text1_map = uppers1 + names1 + lowers1
train_text2_map = uppers2 + names2 + lowers2
train_text3_map = uppers3 + names3 + lowers3
train_text4_map = uppers4 + names4 + lowers4
train_text5_map = uppers5 + names5 + lowers5
train_text6_map = uppers6 + names6 + lowers6

In [81]:
def to_csv(map_list, filename):
    df = pd.DataFrame(map_list, columns=['id', 'word'])
    df = df.sort_values('id').set_index('id')
    df['second word'] = df['word'].shift(-1)
    df['2 words'] = df['word'] + ' ' + df['second word']
    df = df.drop(columns=['word', 'second word'])
    dps = df.duplicated()
    duplicate_rows = df[dps]
    duplicate_rows = duplicate_rows.drop_duplicates()
    duplicate_rows.to_csv(filename)
    return duplicate_rows

In [82]:
def to_csv2(map_list, filename):
    df = pd.DataFrame(map_list, columns=['id', 'word'])
    df = df.sort_values('id').set_index('id')
    df['second word'] = df['word'].shift(-1)
    df['2 words'] = df['word'] + ' ' + df['second word']
    df = df.drop(columns=['word', 'second word'])
    df.to_csv(filename)
    return df

In [83]:
train1_df = to_csv2(train_text1_map, './db/train_text1')
train2_df = to_csv2(train_text2_map, './db/train_text2')
train3_df = to_csv2(train_text3_map, './db/train_text3')
train4_df = to_csv2(train_text4_map, './db/train_text4')
train5_df = to_csv2(train_text5_map, './db/train_text5')
train6_df = to_csv2(train_text6_map, './db/train_text6')

In [84]:
def split_list(any_list):
    splited_list = []

    for line in any_list:
        line = line.split()
        for word in line:
            splited_list.append(word)
    
    return splited_list

In [85]:
game_engine = ['game engine',
               'mobile games',
               'Unity 3D',
               'game engines',
               'engine Unity',
               'mobile games',
               'Unity UDK',
               'levels design',
               'Torque 2D3D']

environment = ['climatic ecological', 
               'state environment', 
               'observations state', 
               'air pollution', 
               'state atmospheric air', 
               'observations carried', 
               'air carried', 
               'RSE Kazhydromet', 
               'air quality', 
               'dust meter', 
               'pollution control', 
               'Atmospheric pollution', 
               'emissions pollutants', 
               'current state']

physics = ['physical processes',
           'physical laboratory',
           'physics sections',
           'real environment',
           'physical experiments',
           'physical laboratories',
           'general physics',
           'Diffraction electrons',
           'Physical experiment',
           'speed bullet',
           'Determination coefficient',
           'Measurement speed',
           'Verification Malus',
           'polarized light',
           '3D models']

robotics = ['using controller',
            'sensitivity receiver',
            'LoRa technology']

it_list = ['levels design',
           'create software',
           'user interface',
           'programming language',
           'software complex',
           'startype network',
           'mobile communication',
           'local station',
           'base station',
           'through web',
           'network fragment',
           'based ARM',
           'parameters electronic',
           'electronic processing',
           'processing devices',
           'MIT Media',
           'LPWAN technology',
           'IoT applications',
           'LPWAN technologies',
           'wireless technologies']

education = ['teaching physics',   
             'laboratory works',    
             'academic programs',    
             'physics course',    
             'technical academic',    
             'laboratory work',    
             'educational software',    
             'virtual laboratories',  
             'Associate professor',
             'Mathematical modelling',
             'Phd candidate',
             'Data analysis',
             'Mathematics School',
             'good example',
             'computer-based learning',    
             'learning systems',    
             'secondary schools',    
             'educational institutions',
             'learning tools',
             'learning process',
             'point view',
             'educational process',
             'medical education',
             'high level knowledge',
             'teaching methods',
             'educational games']


In [86]:
game_engine = split_list(game_engine)
environment = split_list(environment)
physics = split_list(physics)
robotics = split_list(robotics)
it_list = split_list(it_list)
education = split_list(education)

In [97]:
education

['teaching',
 'physics',
 'laboratory',
 'works',
 'academic',
 'programs',
 'physics',
 'course',
 'technical',
 'academic',
 'laboratory',
 'work',
 'educational',
 'software',
 'virtual',
 'laboratories',
 'Associate',
 'professor',
 'Mathematical',
 'modelling',
 'Phd',
 'candidate',
 'Data',
 'analysis',
 'Mathematics',
 'School',
 'good',
 'example',
 'computer-based',
 'learning',
 'learning',
 'systems',
 'secondary',
 'schools',
 'educational',
 'institutions',
 'learning',
 'tools',
 'learning',
 'process',
 'point',
 'view',
 'educational',
 'process',
 'medical',
 'education',
 'high',
 'level',
 'knowledge',
 'teaching',
 'methods',
 'educational',
 'games']

In [88]:
all_df = to_csv(all_words, './db/all')

In [89]:
is_in_ge = all_df['2 words'].isin(game_engine)
is_in_env = all_df['2 words'].isin(environment)
is_in_ph = all_df['2 words'].isin(physics)
is_in_rb = all_df['2 words'].isin(robotics)
is_in_it = all_df['2 words'].isin(it_list)
is_in_edu = all_df['2 words'].isin(education)

if (is_in_ge == True).any():
    all_df.loc[is_in_ge, 'type'] = 'game engine'
if (is_in_env == True).any():
    all_df.loc[is_in_env, 'type'] = 'environment'
if (is_in_ph == True).any():
    all_df.loc[is_in_ph, 'type'] = 'physics'
if (is_in_rb == True).any():
    all_df.loc[is_in_rb, 'type'] = 'robotics'
if (is_in_it == True).any():
    all_df.loc[is_in_it, 'type'] = 'IT'
if (is_in_edu == True).any():
    all_df.loc[is_in_edu, 'type'] = 'education'

In [90]:
def circle_df(train_df):
    train_df.loc[train_df['2 words'].isin(all_df['2 words']), 'type'] = all_df['type'].fillna('unknown')
    type_counts = train_df['type'].value_counts()
    pie_chart = type_counts.plot.pie(figsize=(5, 5))
    return plt.show()

In [91]:
all_df = all_df.dropna()
circle_df(all_df)

KeyError: 'type'

In [92]:
train1_df = train1_df.dropna()
circle_df(train1_df)

KeyError: 'type'

In [ ]:
ge_words = all_df.loc[all_df['type'] == 'game engine', '2 words']
env_words = all_df.loc[all_df['type'] == 'environment', '2 words']
ph_words = all_df.loc[all_df['type'] == 'physics', '2 words']
rb_words = all_df.loc[all_df['type'] == 'robotics', '2 words']
it_words = all_df.loc[all_df['type'] == 'IT', '2 words']
edu_words = all_df.loc[all_df['type'] == 'education', '2 words']

In [ ]:
def get_list(train_df):
    any_list = train_df.tolist()
    return any_list

In [ ]:
def get_list2(train_df):
    any_list = train_df['2 words'].tolist()
    return any_list

In [ ]:
class1_list = get_list(ge_words)
class2_list = get_list(env_words)
class3_list = get_list(ph_words)
class4_list = get_list(rb_words)
class5_list = get_list(it_words)
class6_list = get_list(edu_words)

In [ ]:
class1_list

['game engine',
 'Unity 3D',
 'game engines',
 'engine Unity',
 'mobile games',
 'Unity UDK',
 'Torque 2D3D']

In [ ]:
train1_words = get_list2(train1_df)
train2_words = get_list2(train2_df)
train3_words = get_list2(train3_df)
train4_words = get_list2(train4_df)
train5_words = get_list2(train5_df)
train6_words = get_list2(train6_df)

In [ ]:
def result(train_words):
    class1_count = 0
    class2_count = 0
    class3_count = 0
    class4_count = 0
    class5_count = 0
    class6_count = 0

    for word in train_words:
        if word in class1_list:
            class1_count += 1
        elif word in class2_list:
            class2_count += 1
        elif word in class3_list:
            class3_count += 1
        elif word in class4_list:
            class4_count += 1
        elif word in class5_list:
            class5_count += 1
        elif word in class6_list:
            class6_count += 1
    
    class1_count = class1_count / len(class1_list)
    class2_count = class2_count / len(class2_list)
    class3_count = class3_count / len(class3_list)
    class4_count = class4_count / len(class4_list)
    class5_count = class5_count / len(class5_list)
    class6_count = class6_count / len(class6_list)

    print(class1_count, class2_count, class3_count, class4_count, class5_count, class6_count)
    
    max_class = max(class1_count, class2_count, class3_count, class4_count, class5_count, class6_count)

    if class1_count == max_class:
        return 'Game Engine'
    elif class2_count == max_class:
        return 'Environment'
    elif class3_count == max_class:
        return 'Physics'
    elif class4_count == max_class:
        return 'Robotics'
    elif class5_count == max_class:
        return 'IT'
    elif class6_count == max_class:
        return 'Education'

In [ ]:
result(train1_words)

2.5714285714285716 0.0 0.0 0.0 0.0 0.0


'Game Engine'

In [ ]:
result(train2_words)

0.0 0.0 0.0 0.0 0.0 0.0


'Game Engine'

In [ ]:
result(train3_words)

0.0 0.0 0.0 0.0 0.0 0.0


'Game Engine'

In [ ]:
result(train4_words)

0.0 0.0 0.0 0.0 0.0 0.0


'Game Engine'

In [ ]:
result(train5_words)

0.0 0.0 0.0 0.0 0.0 0.0


'Game Engine'

In [ ]:
result(train6_words)

0.0 0.0 0.0 0.0 0.0 0.0


'Game Engine'